<a href="https://colab.research.google.com/github/NeemaNdanu/Natural-language-processing/blob/main/Parsing_Assign_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Goal: Implement two fundamental parsing algorithms—Cocke-Kasami-Younger (CKY) algorithm and Earley’s algorithm—for parsing sentences based on a context-free grammar (CFG).**

Task 1: Use the following pseudocode structure to implement CKY:

**1. Convert the given CFG into Chomsky Normal Form (CNF).**

The CKY algorithm is a bottom-up parsing method that operates on grammars in Chomsky Normal Form (CNF).

It works by dividing the input sentence into substrings, filling a 2D table based on possible combinations of grammar rules, and constructing a parse tree by checking for possible non-terminal symbols that can derive the sentence.

The CKY algorithm requires the CFG to be in Chomsky Normal Form (CNF). CNF rules are of the form:

* A –> BC, [ with at most two non-terminal symbols on the RHS ]
* A –> a, or [ one terminal symbol on the RHS ]
* S –> nullstring, [ null string ]

Non-terminal symbols are placeholders in a grammar that represent structures like sentences (`S`), noun phrases (`NP`), and verb phrases (`VP`). They can be expanded into sequences of other non-terminals or terminal symbols, which are the actual words in a sentence, such as determiners ("the"), nouns ("cat"), and verbs ("chased").

In [ ]:
# Load the necessary libraries
import copy
from collections import defaultdict

# Define the Context-Free Grammar
grammar = {
    'S': [['NP', 'VP']],
    'NP': [['Det', 'N'], ['Det', 'N', 'PP']],
    'VP': [['V', 'NP'], ['V', 'NP', 'PP']],
    'PP': [['P', 'NP']],
    'Det': [['the'], ['a']],
    'N': [['cat'], ['mouse'], ['garden'], ['police']],
    'V': [['chased'], ['saw'], ['teargassed']],
    'P': [['in']]
}

# Convert CFG to CNF
def is_terminal(symbol):
    return symbol.islower()

def convert_to_cnf(grammar):
    cnf_grammar = copy.deepcopy(grammar)

    # Remove unit productions
    unit_rules = {}
    for lhs in cnf_grammar:
        for rhs in cnf_grammar[lhs]:
            if len(rhs) == 1 and not is_terminal(rhs[0]):
                unit_rules.setdefault(lhs, set()).add(rhs[0])

    while unit_rules:
        lhs, rhs_set = unit_rules.popitem()
        for rhs in rhs_set:
            for production in cnf_grammar.get(rhs, []):
                if len(production) == 1 and not is_terminal(production[0]):
                    unit_rules.setdefault(lhs, set()).add(production[0])
                else:
                    cnf_grammar[lhs].append(production)

    # Convert terminals in rules with length > 1 to non-terminals
    for lhs in list(cnf_grammar.keys()):
        new_productions = []
        for rhs in cnf_grammar[lhs]:
            new_rhs = []
            for symbol in rhs:
                if is_terminal(symbol) and len(rhs) > 1:
                    new_nt = f"{symbol.upper()}_T"
                    if new_nt not in cnf_grammar:
                        cnf_grammar[new_nt] = [[symbol]]
                    new_rhs.append(new_nt)
                else:
                    new_rhs.append(symbol)
            new_productions.append(new_rhs)
        cnf_grammar[lhs] = new_productions

    # Break down rules with more than 2 symbols
    for lhs in list(cnf_grammar.keys()):
        new_productions = []
        for rhs in cnf_grammar[lhs]:
            if len(rhs) > 2:
                current = rhs[0]
                for i in range(1, len(rhs) - 1):
                    new_nt = f"{lhs}_{'_'.join(rhs[:i+1])}"
                    if new_nt not in cnf_grammar:
                        cnf_grammar[new_nt] = [[current, rhs[i]]]
                    current = new_nt
                new_productions.append([current, rhs[-1]])
            else:
                new_productions.append(rhs)
        cnf_grammar[lhs] = new_productions

    return cnf_grammar

# Convert CFG to CNF
cnf_grammar = convert_to_cnf(grammar)
print("CNF Grammar:")
for lhs, productions in cnf_grammar.items():
    for rhs in productions:
        print(f"{lhs} -> {' '.join(rhs)}")



CNF Grammar:
S -> NP VP
NP -> Det N
NP -> NP_Det_N PP
VP -> V NP
VP -> VP_V_NP PP
PP -> P NP
Det -> the
Det -> a
N -> cat
N -> mouse
N -> garden
N -> police
V -> chased
V -> saw
V -> teargassed
P -> in
NP_Det_N -> Det N
VP_V_NP -> V NP


**Explanation:**

Let's break down the output to confirm if it has been successfully converted to **Chomsky Normal Form (CNF):** Below is a few of the rules:-

1. **Start Symbol Rule**

- **S -> NP VP**

  - The sentence (S) is composed of a noun phrase (NP) followed by a verb phrase (VP), reflecting the typical subject-verb-object structure in English.

2. **Noun Phrase (NP) Rules**

- **NP -> Det N**

  - A noun phrase consists of a determiner (Det) and a noun (N), as in simple phrases like "the cat."

- **NP -> NP_Det_N PP**

  - A more complex noun phrase can include a simpler noun phrase (NP_Det_N) followed by a prepositional phrase (PP), allowing for phrases like "the cat in the garden."

3. **Verb Phrase (VP) Rules**

- **VP -> V NP**

  - A verb phrase consists of a verb (V) followed by a noun phrase (NP), covering structures like "chased the mouse."

Let's breaking down the sentence:

- The sentence starts with **S -> NP VP**:

  - The noun phrase ("the cat") is constructed from **NP -> Det N**, where **Det -> "the"** and **N -> "cat"**.

  - The verb phrase ("chased a mouse in the garden") follows **VP -> VP_V_NP PP**:

    - **VP_V_NP -> V NP** builds "chased a mouse" with **V -> "chased"** and **NP -> "a mouse"**.

    - The prepositional phrase "in the garden" is constructed using **PP -> P NP**, where **P -> "in"** and **NP -> "the garden"**.

The output follows **Chomsky Normal Form (CNF)** because each rule either:

1. Produces two non-terminals (e.g., **S -> NP VP**).

2. Produces one terminal (e.g., **Det -> "the"**).

This confirms the sentence is successfully converted into CNF.


* **2.	Create a 2D table to represent substrings of the input sentence.**

In [ ]:
# Initialize the CKY parser
def initialize_table(sentence_length):
    return [[set() for _ in range(sentence_length + 1)] for _ in range(sentence_length)]

* **3.	Iterate over substrings and fill in the table based on grammar rules.**

In [ ]:
!pip install prettytable

In [ ]:
# Iterate over substrings and fill in the table based on grammar rules
from prettytable import PrettyTable

def cky_parse(sentence, grammar):
    words = sentence.split()
    n = len(words)
    table = initialize_table(n)

    # Initialize the table for terminal symbols
    for i, word in enumerate(words):
        for lhs in grammar:
            for rhs in grammar[lhs]:
                if len(rhs) == 1 and rhs[0] == word:
                    table[i][i + 1].add(lhs)

    # Fill the table for spans greater than 1
    for span in range(2, n + 1):
        for begin in range(n - span + 1):
            end = begin + span
            for split in range(begin + 1, end):
                left_cell = table[begin][split]
                right_cell = table[split][end]
                for lhs in grammar:
                    for rhs in grammar[lhs]:
                        if len(rhs) == 2 and rhs[0] in left_cell and rhs[1] in right_cell:
                            table[begin][end].add(lhs)

    # Create a pretty table
    pretty_table = PrettyTable()
    headers = ["Span"] + [f"{i}" for i in range(n + 1)]
    pretty_table.field_names = headers

    for i in range(n):
        row = [f"{i}-{i+1}"] + [' | '.join(sorted(cell)) if cell else '-' for cell in table[i]]
        pretty_table.add_row(row)

    print("CKY Table:")
    print(pretty_table)
    print()

    return 'S' in table[0][n], table

* **4.	Return whether the input sentence is valid or not according to the CFG**

In [ ]:
# Test sentences
sentences = [
    "the cat chased the mouse",
    " the cat was on the table",
    "the cat saw the mouse",
    "the mouse is after the cat"
]

# Parse and check validity of each sentence
for sentence in sentences:
    is_valid, table = cky_parse(sentence, cnf_grammar)
    print(f"Sentence: '{sentence}' is {'valid' if is_valid else 'invalid'} according to the grammar.\n")


CKY Table:
+------+---+-----+---------------+---+-----+---------------+
| Span | 0 |  1  |       2       | 3 |  4  |       5       |
+------+---+-----+---------------+---+-----+---------------+
| 0-1  | - | Det | NP | NP_Det_N | - |  -  |       S       |
| 1-2  | - |  -  |       N       | - |  -  |       -       |
| 2-3  | - |  -  |       -       | V |  -  |  VP | VP_V_NP |
| 3-4  | - |  -  |       -       | - | Det | NP | NP_Det_N |
| 4-5  | - |  -  |       -       | - |  -  |       N       |
+------+---+-----+---------------+---+-----+---------------+

Sentence: 'the cat chased the mouse' is valid according to the grammar.

CKY Table:
+------+---+-----+---------------+---+---+-----+---+
| Span | 0 |  1  |       2       | 3 | 4 |  5  | 6 |
+------+---+-----+---------------+---+---+-----+---+
| 0-1  | - | Det | NP | NP_Det_N | - | - |  -  | - |
| 1-2  | - |  -  |       N       | - | - |  -  | - |
| 2-3  | - |  -  |       -       | - | - |  -  | - |
| 3-4  | - |  -  |       -       | - 

**Explanation**

To understand why some sentences are valid and others are not, and how the CKY algorithm works, here’s a breakdown:

For the first sentence, "the cat chased the mouse," let’s look at why it is valid. Each column in the CKY table corresponds to a word in the sentence:

- Word 0: "the" (Det)
- Word 1: "cat" (N)
- Word 2: "chased" (V)
- Word 3: "the" (Det)
- Word 4: "mouse" (N)

Here’s how they are broken down:

- Span 0-1 ("the"): Det → "the", matching the rule Det -> the.
- Span 1-2 ("cat"): N → "cat", matching the rule N -> cat. Together with "the" (Det), they form a noun phrase (NP).
- Span 2-3 ("chased"): V → "chased", matching the rule V -> chased. This creates a verb phrase (VP).
- Span 3-4 ("the"): Det → "the", matching the rule Det -> the.
- Span 4-5 ("mouse"): N → "mouse", matching the rule N -> mouse. Together with "the" (Det), they form another noun phrase (NP).

Spanning the sentence:

- Spans 0-2 form a valid noun phrase (NP) with the rule NP -> Det N.
- Spans 2-5 form a valid verb phrase (VP) with the rule VP -> V NP, allowing a complete sentence with S -> NP VP.

**Conclusion**: The CKY table confirms that "the cat chased the mouse" is valid according to the grammar.

For the second sentence, "the cat was on the table," the word segmentation is:

- Word 0: "the" (Det)
- Word 1: "cat" (N)
- Word 2: "was" (V, but not in the grammar)
- Word 3: "on" (P)
- Word 4: "the" (Det)
- Word 5: "table" (N)

The CKY table shows that the parser fails to recognize "was" and "on" because there are no production rules for these words in the grammar. As a result:

- Span 2-3 ("was") does not match any rule.
- Span 3-4 ("on") is a preposition (P), but it cannot connect to a valid structure (like a prepositional phrase).

**Conclusion**: The sentence "the cat was on the table" is invalid because the CKY algorithm cannot form a valid parse tree due to missing rules for "was" and "on".

**Task 2: Use the following pseudocode structure to implement Earley’s algorithm:**
 * **1.	Initialize the Earley chart for storing parsing states.**


The Earley algorithm is a flexible top-down parsing method that can handle arbitrary CFGs without the need for CNF conversion.

It uses prediction, scanning, and completion operations to build parsing states as it reads through the input sentence, updating a chart at each position.

**Key Components:**
- **Position (0, 1, 2, …):** Represents the word positions in the sentence, where Position 0 is before the first word, Position 1 is after the first word, and so on.
- **Rules:** Grammar rules are tracked, such as S -> NP VP or NP -> Det N. A dot (•) is used to show which part of the rule has been matched. For instance, NP -> Det • N means the determiner has been matched, and the parser now expects a noun.

**How it Works:**
1. **Initialization (Position 0):** At the start, the parser predicts all possible rules the sentence could begin with, such as S -> • NP VP. It expects the sentence to start with an NP (noun phrase), which may begin with a determiner (Det), like "the" or "a."
  
2. **Scanning & Predicting (Position 1 and beyond):** As the parser scans the words, it updates the rules. For example, after seeing "the" at Position 1, it updates the rule to NP -> Det • N, indicating the determiner is matched, and it's now looking for a noun.

3. **Complete Matches:** When a rule is fully matched (dot reaches the end), the parser completes it. For example, after matching NP -> Det N, it checks what rule predicted NP and continues parsing from there.



In [ ]:
import nltk
from nltk import CFG
from collections import defaultdict
from prettytable import PrettyTable

# Define the CFG using NLTK
grammar = CFG.fromstring("""
    S -> NP VP
    VP -> V NP | V NP PP
    PP -> P NP
    NP -> Det N | Det N PP
    Det -> 'the' | 'a'
    N -> 'cat' | 'mouse' | 'garden' | 'police' | 'tree' | 'bird'
    V -> 'chased' | 'saw' | 'climbed' | 'catch'
    P -> 'in' | 'over' | 'to'
""")

# State class
class State:
    def __init__(self, lhs, rhs, dot, start):
        # Left-hand side (non-terminal)
        self.lhs = lhs
        # Right-hand side (list)
        self.rhs = rhs
        # Position of the dot
        self.dot = dot
        # Start position in the input
        self.start = start

    def is_complete(self):
        return self.dot >= len(self.rhs)

    def next_symbol(self):
        if self.is_complete():
            return None
        return self.rhs[self.dot]

    def __eq__(self, other):
        return (self.lhs, self.rhs, self.dot, self.start) == (other.lhs, other.rhs, other.dot, other.start)

    def __hash__(self):
        return hash((self.lhs, tuple(self.rhs), self.dot, self.start))

    def __repr__(self):
        before_dot = ' '.join(str(x) for x in self.rhs[:self.dot])
        after_dot = ' '.join(str(x) for x in self.rhs[self.dot:])
        return f"{self.lhs} -> {before_dot} • {after_dot} [{self.start}]"


* **2.	Iterate over input tokens and apply the predict, scan, and complete operations.**


In [ ]:
# Earley Parser Class
class EarleyParserCustom:
    def __init__(self, grammar):
        self.grammar = grammar
        self.rules = defaultdict(list)
        for production in grammar.productions():
            self.rules[production.lhs()].append(production.rhs())
        self.chart = []

    def predictor(self, state, chart, index):
        symbol = state.next_symbol()
        for production in self.rules[symbol]:
            new_state = State(symbol, production, 0, index)
            if new_state not in chart[index]:
                chart[index].add(new_state)

    def scanner(self, state, chart, index, tokens):
        symbol = state.next_symbol()
        if index < len(tokens):
            word = tokens[index]
            if word == symbol:
                new_state = State(state.lhs, state.rhs, state.dot + 1, state.start)
                if new_state not in chart[index + 1]:
                    chart[index + 1].add(new_state)

    def completer(self, state, chart, index):
        for prev_state in chart[state.start]:
            if not prev_state.is_complete() and prev_state.next_symbol() == state.lhs:
                new_state = State(prev_state.lhs, prev_state.rhs, prev_state.dot + 1, prev_state.start)
                if new_state not in chart[index]:
                    chart[index].add(new_state)

    def parse(self, tokens):
        n = len(tokens)
        self.chart = [set() for _ in range(n + 1)]
        start_productions = self.rules[self.grammar.start()]
        for prod in start_productions:
            start_state = State(self.grammar.start(), prod, 0, 0)
            self.chart[0].add(start_state)

        for i in range(n + 1):
            added = True
            while added:
                added = False
                states = list(self.chart[i])
                for state in states:
                    if not state.is_complete():
                        symbol = state.next_symbol()
                        if isinstance(symbol, nltk.grammar.Nonterminal):
                            # Predictor
                            before = len(self.chart[i])
                            self.predictor(state, self.chart, i)
                            after = len(self.chart[i])
                            if after > before:
                                added = True
                        else:
                            # Scanner
                            before = len(self.chart[i + 1] if i < n else [])
                            self.scanner(state, self.chart, i, tokens)
                            after = len(self.chart[i + 1] if i < n else [])
                            if i < n and after > before:
                                added = True
                    else:
                        # Completer
                        before = len(self.chart[i])
                        self.completer(state, self.chart, i)
                        after = len(self.chart[i])
                        if after > before:
                            added = True

        return self.is_parsed()

    def is_parsed(self):
        for state in self.chart[-1]:
            if (state.lhs == self.grammar.start() and state.is_complete() and state.start == 0):
                return True
        return False

    def display_chart(self, tokens):
        pretty_table = PrettyTable()
        headers = ["Position"] + [f"{i}" for i in range(len(tokens) + 1)]
        pretty_table.field_names = headers

        for i in range(len(tokens) + 1):
            states = '\n'.join(str(state) for state in sorted(self.chart[i], key=lambda s: (s.lhs, s.rhs, s.dot, s.start)))
            pretty_table.add_row([f"{i}"] + [states if j == i else '' for j in range(len(tokens) + 1)])

        print("Earley Chart:")
        print(pretty_table)
        print()


* **3.	Parse the input sentence and check if it matches the grammar.**


In [ ]:
# Testing the Earley Parser
def earley_parse(sentence, grammar):
    tokens = sentence.split()
    parser = EarleyParserCustom(grammar)
    is_valid = parser.parse(tokens)
    parser.display_chart(tokens)
    return is_valid

# Test Sentences
sentences = [
    "the cat chased the mouse",
    "the cat was on the table",
    "the cat saw the mouse",
    "the mouse is after cat",
]

# Parsing the Sentences
for sentence in sentences:
    print(f"Parsing Sentence: '{sentence}'")
    is_valid = earley_parse(sentence, grammar)
    print(f"Result: The sentence is {'valid' if is_valid else 'invalid'} according to the grammar.\n")

Parsing Sentence: 'the cat chased the mouse'
Earley Chart:
+----------+-----------------------+----------------------+----------------------+-----------------------+----------------------+----------------------+
| Position |           0           |          1           |          2           |           3           |          4           |          5           |
+----------+-----------------------+----------------------+----------------------+-----------------------+----------------------+----------------------+
|    0     |    Det ->  • a [0]    |                      |                      |                       |                      |                      |
|          |   Det ->  • the [0]   |                      |                      |                       |                      |                      |
|          |   NP ->  • Det N [0]  |                      |                      |                       |                      |                      |
|          | NP ->  • D

***Explanation:**

Based on how parsing works, here's why one sentence is valid and the other is not:

**Sentence 1: "the cat chased the mouse"**
The parser successfully builds this sentence using the grammar:

- **Noun Phrase (NP):** It parses "the" (determiner) followed by "cat" (noun) to form a noun phrase (NP).
  
- **Verb Phrase (VP):** It parses "chased" (verb) followed by the noun phrase "the mouse" (object).

Finally, the parser combines the noun phrase (subject) and the verb phrase to form a complete sentence. The final rule, **S -> NP VP •**, is fully matched, indicating the sentence is valid according to the grammar.

 **Sentence 2: "the cat was on the table"**
This sentence is **invalid** because the grammar doesn't support certain elements:

- The parser recognizes "the" and "cat" as a noun phrase (NP).
  
- However, when it encounters "was" at position 3, it fails to match any verb (V) rule since the grammar likely expects a simple verb like "chased." Additionally, it cannot handle the prepositional phrase "on the table."

This causes the parser to stop without completing the sentence, marking it as **invalid**.


**Task 3:**

* **1.	Try parsing complex sentences or experiment with different CFGs.**



In [ ]:
# Test complex sentences
complex_sentences = [
    "the police chased the man in the garden",
    "the boy saw the girl with the telescope",
    "the police chased the cat in the garden",
]

# Parse and check validity of each sentence using CKY
for sentence in complex_sentences:
    is_valid, table = cky_parse(sentence, cnf_grammar)
    print(f"Sentence: '{sentence}' is {'valid' if is_valid else 'invalid'} according to the grammar.\n")


CKY Table:
+------+---+-----+---------------+---+-----+---+---+-----+---------------+
| Span | 0 |  1  |       2       | 3 |  4  | 5 | 6 |  7  |       8       |
+------+---+-----+---------------+---+-----+---+---+-----+---------------+
| 0-1  | - | Det | NP | NP_Det_N | - |  -  | - | - |  -  |       -       |
| 1-2  | - |  -  |       N       | - |  -  | - | - |  -  |       -       |
| 2-3  | - |  -  |       -       | V |  -  | - | - |  -  |       -       |
| 3-4  | - |  -  |       -       | - | Det | - | - |  -  |       -       |
| 4-5  | - |  -  |       -       | - |  -  | - | - |  -  |       -       |
| 5-6  | - |  -  |       -       | - |  -  | - | P |  -  |       PP      |
| 6-7  | - |  -  |       -       | - |  -  | - | - | Det | NP | NP_Det_N |
| 7-8  | - |  -  |       -       | - |  -  | - | - |  -  |       N       |
+------+---+-----+---------------+---+-----+---+---+-----+---------------+

Sentence: 'the police chased the man in the garden' is invalid according to the grammar.

In [ ]:
# Parse complex sentences using Earley
for sentence in complex_sentences:
    print(f"Parsing Sentence: '{sentence}'")
    is_valid = earley_parse(sentence, grammar)
    print(f"Result: The sentence is {'valid' if is_valid else 'invalid'} according to the grammar.\n")


Parsing Sentence: 'the police chased the man in the garden'
Earley Chart:
+----------+-----------------------+----------------------+----------------------+-----------------------+----------------------+---+---+---+---+
| Position |           0           |          1           |          2           |           3           |          4           | 5 | 6 | 7 | 8 |
+----------+-----------------------+----------------------+----------------------+-----------------------+----------------------+---+---+---+---+
|    0     |    Det ->  • a [0]    |                      |                      |                       |                      |   |   |   |   |
|          |   Det ->  • the [0]   |                      |                      |                       |                      |   |   |   |   |
|          |   NP ->  • Det N [0]  |                      |                      |                       |                      |   |   |   |   |
|          | NP ->  • Det N PP [0] |              

**EXPLANATION**

 If we use more complex sentences like "the boy saw the girl with the telescope" or "the police chased the cat in the garden," here's what happens:

**Sentence 1: "the boy saw the girl with the telescope"**

* This sentence would be **invalid** under the current grammar (whether using CKY or Earley’s Algorithm) because the grammar does not include a rule for the preposition "with."

* Specifically, the phrase "with the telescope" introduces a prepositional phrase, but since we haven't defined a rule that allows "with" as a valid preposition, the parser won't recognize it.

* To handle this sentence, we would need to modify the grammar to include a rule for prepositional phrases with "with" (e.g., **PP -> P NP**, where **P -> "with"**).

**Sentence 2: "the police chased the cat in the garden"**

* This sentence would be **valid** because the grammar includes rules for handling prepositional phrases like "in the garden."

* The parser can recognize "the police" as a noun phrase (NP), "chased" as a verb (V), and "the cat in the garden" as a combination of a noun phrase (NP) and a prepositional phrase (PP), making it fully parseable under both CKY and Earley's Algorithm.

In short, the first sentence is invalid due to the absence of a rule for "with," while the second sentence is valid because the grammar supports prepositional phrases like "in the garden." Adjusting the grammar would be necessary to parse more complex structures like "with the telescope."


* **2.	Compare the performance and efficiency of CKY vs. Earley when handling ambiguous or complex sentences.**

When comparing **CKY** and **Earley** parsers based on performance and efficiency:

**Performance Metrics:**

**CKY:**
- **Time Complexity:** O(n³) – slower for long sentences.
- **Space Complexity:** O(n²).
- **Performance:** Works well for unambiguous or less complex grammars, but faces challenges with larger or ambiguous sentences due to its cubic time complexity.

**Earley:**
- **Time Complexity:** Worst-case O(n³), but typically O(n²) for unambiguous grammars and O(n) for most LR(k) grammars.
- **Space Complexity:** O(n²).
- **Performance:** More efficient in practice, especially with unambiguous or simple grammars, as it reduces unnecessary computations.

**When handling ambiguity and complexity:**

**CKY:**
- Designed for CNF grammars, requiring conversion if not in this form.
- Can handle ambiguity but struggles with multiple parse trees, leading to resource-intensive management.
- Not suitable for left-recursive grammars unless transformed.

**Earley:**
- Can handle any CFG, including ambiguous and left-recursive grammars.
- Better at managing ambiguity and multiple parse trees without significant performance degradation.
- The predictor, scanner, and completer operations offer flexibility in parsing strategies.

**When implementating complexity:**

**CKY:**
- Simpler to implement once the grammar is converted to CNF, but the conversion step adds complexity.
  
**Earley:**
- More complex to implement due to dynamic operations and state tracking but doesn't require grammar conversion, making it more flexible for a broader range of grammars.

**In conclusion:**
- **CKY** is simpler but less flexible, particularly for complex or ambiguous grammars.
- **Earley** is more adaptable, especially for ambiguous sentences, and often more efficient in practical cases despite being harder to implement.



**Task 4: Provide a lab report documenting the results and performance of the parsing algorithms. (Include this at the bottom of the notebook)**

* **Explain the output for multiple test sentences parsed using CKY and Earley.**


**With Simple Sentences:**

**Test 1: "The cat chased the mouse"**

- **CKY:** The sentence is valid. CKY successfully parses the sentence by breaking it into components: the noun phrase (NP) "the cat" and the verb phrase (VP) "chased the mouse." The CKY table clearly displays these structures, with spans that match the grammar rules.
  
- **Earley:** The Earley parser also parses the sentence correctly. It predicts, scans, and completes the rules as expected. Earley's flexibility with arbitrary CFGs means it handles the sentence without needing CNF conversion.

**Test 2: "The cat was on the table"**

- **CKY:** The sentence is invalid because "was" and "on" do not appear in the CNF grammar. CKY cannot match these words to any rules, causing the parsing process to fail.

- **Earley:** Similar to CKY, Earley fails to parse the sentence since "was" is not predicted by any grammar rule. However, Earley's chart shows exactly where the parsing failed, making it clear where the process stopped.


**With Complex Sentences:**

**Test 1: "The boy saw the girl with the telescope"**

- **CKY:** The sentence is invalid because CKY, using the current grammar, lacks a rule for prepositional phrases like "with the telescope." The table cannot be fully populated, leaving the sentence incomplete.

- **Earley:** Earley also fails to parse the sentence for the same reason: the grammar does not predict the prepositional phrase (PP) "with the telescope." However, Earley’s chart will indicate where the parse stopped at the preposition "with."

**Test 2: "The police chased the cat in the garden"**

- **CKY:** The sentence is valid under CKY because the grammar includes rules for handling prepositional phrases like "in the garden." CKY successfully identifies the noun phrase (NP) "the police," the verb phrase (VP) "chased the cat," and the prepositional phrase (PP) "in the garden."

- **Earley:** The sentence is also valid under Earley. It predicts and completes the necessary rules for the prepositional phrase "in the garden," producing a complete parse. The Earley chart shows each step of the parsing process, demonstrating how the sentence is successfully parsed.

*	**Discuss the limitations of each algorithm and where one might be preferred over the other.**



**Weaknesses of CKY Algorithm:**
1. **Grammar Restriction:** CKY works only with grammars in Chomsky Normal Form (CNF), which adds complexity during grammar conversion and reduces flexibility.
2. **Ambiguity Handling:** CKY struggles with ambiguous grammars, as it doesn’t naturally provide multiple parse trees without additional effort.

**Weaknesses of Earley’s Algorithm:**
1. **Overhead for Simple Grammars:** Earley’s algorithm may be slower than CKY for simple grammars, where the latter is more efficient.
2. **Variable Time Complexity:** Earley’s performance is less predictable, with worst-case time complexity of O(n³) for highly ambiguous grammars.
3. **Complex Implementation:** Earley’s algorithm is more difficult to implement due to its dynamic state management.

**Comparison:**

- **CKY** is ideal for grammars that are either straightforward or can be easily converted to CNF. It’s efficient for parsing simple, unambiguous sentences but struggles with ambiguity or more complex grammars.
  
- **Earley** is better suited for complex and ambiguous grammars, including those with left recursion or arbitrary CFGs. While it’s more versatile and can handle multiple parse trees, it can introduce overhead for simpler cases due to its complexity.

**Summary:**
- **CKY:** Best for simple, unambiguous grammars in CNF; faster for straightforward cases but less adaptable to ambiguity.
- **Earley:** More flexible for complex and ambiguous grammars, though potentially slower and harder to implement for simpler scenarios.
